In [1]:
##### Global Module

import numpy as np
import pandas as pd
import os
import warnings

warnings.simplefilter('ignore')

In [2]:
##### Path and Read

train_df = pd.read_csv('data/train_data.csv')
test_df = pd.read_csv('data/test_features.csv')
print(train_df.shape)
print(test_df.shape)
#train_df.head()

(113, 22)
(33, 21)


In [3]:
##### Tirmming 1

drop_li = ['email_address']
commu_li = ['from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi',
            'shared_receipt_with_poi', 'to_messages']
stock_li = ['exercised_stock_options', 'restricted_stock', 'restricted_stock_deferred']
cash_li  = ['bonus', 'deferral_payments', 'deferred_income', 'director_fees', 'expenses',
            'loan_advances', 'long_term_incentive', 'other', 'salary']
total_li = ['total_payments', 'total_stock_value']
poi_li   = ['poi']

train_df = train_df.drop(drop_li, axis=1)

##### Trimming 2: weird BELFER ROBERT
weird_idx = train_df[train_df['name'] == 'BELFER ROBERT'].index
train_df = train_df.drop(weird_idx, axis=0)

train_df

,name,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,19794175.0,46950.0,18.0,42.0,4.0,...,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,19250000.0,29336.0,108.0,88.0,30.0,...,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,1624396.0,22884.0,39.0,13.0,14.0,...,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,NaN,118134.0,NaN,NaN,NaN,...,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,NaN,35818.0,144.0,199.0,25.0,...,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,STABLER FRANK,500000.0,NaN,NaN,NaN,NaN,16514.0,NaN,NaN,NaN,...,NaN,356071.0,False,511734.0,NaN,239502.0,NaN,NaN,1112087.0,511734.0
109,BROWN MICHAEL,NaN,NaN,NaN,NaN,NaN,49288.0,41.0,13.0,1.0,...,NaN,NaN,False,NaN,NaN,NaN,761.0,1486.0,49288.0,NaN
110,PICKERING MARK R,300000.0,NaN,NaN,NaN,28798.0,31653.0,67.0,7.0,0.0,...,NaN,NaN,False,NaN,NaN,655037.0,728.0,898.0,1386690.0,28798.0
111,HAUG DAVID L,NaN,NaN,NaN,NaN,NaN,475.0,19.0,4.0,7.0,...,NaN,NaN,False,2217299.0,NaN,NaN,471.0,573.0,475.0,2217299.0


In [4]:
##### Trim commu 1

commu_sub_df = train_df[commu_li]
for i in range(len(commu_li)):
    print(f'{commu_li[i]}: {list(commu_sub_df[commu_li[i]].nlargest(5))}')

from_messages: [14368.0, 6759.0, 4343.0, 2742.0, 2681.0]
from_poi_to_this_person: [528.0, 305.0, 251.0, 240.0, 204.0]
from_this_person_to_poi: [411.0, 387.0, 386.0, 194.0, 194.0]
shared_receipt_with_poi: [4527.0, 3962.0, 3920.0, 3669.0, 3639.0]
to_messages: [15149.0, 12754.0, 8305.0, 7315.0, 7259.0]


In [5]:
##### Trim commu 2: wipe out 'from_messages' outlier

commu_sub_df['from_messages'][commu_sub_df['from_messages'] == commu_sub_df['from_messages'].max()] = np.nan
commu_sub_df = commu_sub_df.fillna(value=commu_sub_df.mean(axis=0, skipna=True), axis = 0)
#commu_sub_df

In [6]:
##### Trim stock 1

stock_sub_df = train_df[stock_li]
print('Big 5')
for i in range(len(stock_li)):
    print(f'{stock_li[i]}: {list(stock_sub_df[stock_li[i]].nlargest(5))}')
print('Small 5')
for i in range(len(stock_li)):
    print(f'{stock_li[i]}: {list(stock_sub_df[stock_li[i]].nsmallest(5))}')

Big 5
exercised_stock_options: [34348384.0, 30766064.0, 19794175.0, 19250000.0, 15364167.0]
restricted_stock: [14761694.0, 13847074.0, 8453763.0, 6843672.0, 4131594.0]
restricted_stock_deferred: [-44093.0, -72419.0, -94556.0, -126027.0, -153686.0]
Small 5
exercised_stock_options: [9803.0, 17378.0, 28798.0, 59539.0, 81042.0]
restricted_stock: [44093.0, 75838.0, 94556.0, 126027.0, 126027.0]
restricted_stock_deferred: [-560222.0, -472568.0, -409554.0, -329825.0, -153686.0]


In [7]:
##### Trim stock 2

stock_sub_df = train_df[stock_li].fillna(value=0)
#stock_sub_df

In [8]:
##### Trim cash 1

cash_sub_df = train_df[cash_li]
print('Big 5')
for i in range(len(cash_li)):
    print(f'{cash_li[i]}: {list(cash_sub_df[cash_li[i]].nlargest(5))}')
print('Small 5')
for i in range(len(cash_li)):
    print(f'{cash_li[i]}: {list(cash_sub_df[cash_li[i]].nsmallest(5))}')

Big 5
bonus: [8000000.0, 7000000.0, 5600000.0, 4175000.0, 3100000.0]
deferral_payments: [2964506.0, 2869717.0, 2157527.0, 1848227.0, 1295738.0]
deferred_income: [-1042.0, -4167.0, -5104.0, -10800.0, -17500.0]
director_fees: [125034.0, 119292.0, 113784.0, 112492.0, 109298.0]
expenses: [228763.0, 228656.0, 178979.0, 137767.0, 127017.0]
loan_advances: [81525000.0, 400000.0]
long_term_incentive: [5145434.0, 3600000.0, 2234774.0, 2035380.0, 1920000.0]
other: [10359729.0, 2818454.0, 2660303.0, 1829457.0, 1573324.0]
salary: [1111258.0, 1072321.0, 655037.0, 510364.0, 440698.0]
Small 5
bonus: [70000.0, 100000.0, 200000.0, 200000.0, 250000.0]
deferral_payments: [7961.0, 10259.0, 16586.0, 27610.0, 50591.0]
deferred_income: [-3504386.0, -3081055.0, -1386055.0, -1386055.0, -934484.0]
director_fees: [17500.0, 36666.0, 101250.0, 102492.0, 103750.0]
expenses: [148.0, 475.0, 600.0, 1279.0, 1413.0]
loan_advances: [400000.0, 81525000.0]
long_term_incentive: [71023.0, 93750.0, 100000.0, 125000.0, 135836.0

In [9]:
##### Trim cash 2: delete 'loan_advances'

del cash_li[5]
cash_sub_df = train_df[cash_li].fillna(value=0)
#cash_sub_df

In [10]:
##### Trim total 1:

train_df['total_payments']    = cash_sub_df.sum(axis = 1)
train_df['total_stock_value'] = stock_sub_df.sum(axis = 1)

In [11]:
x_train_df = pd.concat([train_df['name'], commu_sub_df, stock_sub_df, cash_sub_df, train_df[total_li]], axis=1).set_index('name')
x_train_df

,from_messages,from_poi_to_this_person,from_this_person_to_poi,shared_receipt_with_poi,to_messages,exercised_stock_options,restricted_stock,restricted_stock_deferred,bonus,deferral_payments,deferred_income,director_fees,expenses,long_term_incentive,other,salary,total_payments,total_stock_value
name,,,,,,,,,,,,,,,,,,
RICE KENNETH D,18.0000,42.0,4.000000,864.000000,905.000000,19794175.0,2748364.0,0.0,1750000.0,0.0,-3504386.0,0.0,46950.0,1617011.0,174839.0,420636.0,505050.0,22542539.0
SKILLING JEFFREY K,108.0000,88.0,30.000000,2042.000000,3627.000000,19250000.0,6843672.0,0.0,5600000.0,0.0,0.0,0.0,29336.0,1920000.0,22122.0,1111258.0,8682716.0,26093672.0
SHELBY REX,39.0000,13.0,14.000000,91.000000,225.000000,1624396.0,869220.0,0.0,200000.0,0.0,-4167.0,0.0,22884.0,0.0,1573324.0,211844.0,2003885.0,2493616.0
KOPPER MICHAEL J,497.9375,64.8,40.092308,1111.369231,2156.061538,0.0,985032.0,0.0,800000.0,0.0,0.0,0.0,118134.0,602671.0,907502.0,224305.0,2652612.0,985032.0
CALGER CHRISTOPHER F,144.0000,199.0,25.000000,2188.000000,2598.000000,0.0,126027.0,0.0,1250000.0,0.0,-262500.0,0.0,35818.0,375304.0,486.0,240189.0,1639297.0,126027.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
STABLER FRANK,497.9375,64.8,40.092308,1111.369231,2156.061538,0.0,511734.0,0.0,500000.0,0.0,0.0,0.0,16514.0,0.0,356071.0,239502.0,1112087.0,511734.0
BROWN MICHAEL,41.0000,13.0,1.000000,761.000000,1486.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49288.0,0.0,0.0,0.0,49288.0,0.0
PICKERING MARK R,67.0000,7.0,0.000000,728.000000,898.000000,28798.0,0.0,0.0,300000.0,0.0,0.0,0.0,31653.0,0.0,0.0,655037.0,986690.0,28798.0


In [12]:
y_train_df = pd.concat([train_df['name'], train_df['poi']], axis=1).set_index('name')
y_train_df[y_train_df['poi'] == True] = 1
y_train_df[y_train_df['poi'] == False] = 0
y_train_df

,poi
name,
RICE KENNETH D,1
SKILLING JEFFREY K,1
SHELBY REX,1
KOPPER MICHAEL J,1
CALGER CHRISTOPHER F,1
...,...
STABLER FRANK,0
BROWN MICHAEL,0
PICKERING MARK R,0


In [13]:
#####
##### Test
#####

In [14]:
##### Trim commu 1

commu_sub_df = test_df[commu_li]
for i in range(len(commu_li)):
    print(f'{commu_li[i]}: {list(commu_sub_df[commu_li[i]].nlargest(10))}')

from_messages: [3069.0, 1073.0, 909.0, 484.0, 92.0, 75.0, 49.0, 48.0, 40.0, 32.0]
from_poi_to_this_person: [242.0, 228.0, 188.0, 140.0, 66.0, 64.0, 58.0, 58.0, 44.0, 42.0]
from_this_person_to_poi: [609.0, 108.0, 37.0, 26.0, 23.0, 21.0, 20.0, 18.0, 15.0, 15.0]
shared_receipt_with_poi: [5521.0, 2979.0, 2228.0, 2103.0, 2097.0, 1604.0, 1593.0, 1585.0, 1401.0, 1336.0]
to_messages: [7991.0, 3275.0, 3187.0, 3093.0, 2355.0, 2350.0, 2181.0, 1892.0, 1858.0, 1755.0]


In [15]:
##### Trim commu 2: wipe out 'from_messages' outlier

commu_sub_df = commu_sub_df.fillna(value=commu_sub_df.mean(axis=0, skipna=True), axis = 0)
#commu_sub_df

In [16]:
##### Trim stock 1

stock_sub_df = test_df[stock_li]
print('Big 5')
for i in range(len(stock_li)):
    print(f'{stock_li[i]}: {list(stock_sub_df[stock_li[i]].nlargest(5))}')
print('Small 5')
for i in range(len(stock_li)):
    print(f'{stock_li[i]}: {list(stock_sub_df[stock_li[i]].nsmallest(5))}')

Big 5
exercised_stock_options: [311764000.0, 10433518.0, 8831913.0, 5538001.0, 5210569.0]
restricted_stock: [130322299.0, 4188667.0, 2502063.0, 2046079.0, 1787380.0]
restricted_stock_deferred: [15456290.0, -32460.0, -82782.0, -140264.0, -307301.0]
Small 5
exercised_stock_options: [765920.0, 850477.0, 886231.0, 940257.0, 953136.0]
restricted_stock: [-2604490.0, 32460.0, 47304.0, 145796.0, 157569.0]
restricted_stock_deferred: [-7576788.0, -1787380.0, -315068.0, -307301.0, -140264.0]


In [17]:
##### Trim stock 2: Modify the weird 'BHATNAGAR SANJAY'

test_df.loc[test_df[test_df['name'] == 'BHATNAGAR SANJAY'].index, stock_li] = [0, 2604490, -2604490]
stock_sub_df = test_df[stock_li].fillna(value=0)
#stock_sub_df

In [18]:
##### Trim cash 1

cash_sub_df = test_df[cash_li]
print('Big 5')
for i in range(len(cash_li)):
    print(f'{cash_li[i]}: {list(cash_sub_df[cash_li[i]].nlargest(5))}')
print('Small 5')
for i in range(len(cash_li)):
    print(f'{cash_li[i]}: {list(cash_sub_df[cash_li[i]].nsmallest(5))}')

Big 5
bonus: [97343619.0, 5249999.0, 3000000.0, 2600000.0, 2500000.0]
deferral_payments: [32083396.0, 6426990.0, 3131860.0, 2144013.0, 875307.0]
deferred_income: [-833.0, -38346.0, -51042.0, -98784.0, -159792.0]
director_fees: [1398517.0, 137864.0, 98784.0, 38346.0]
expenses: [5235198.0, 137108.0, 129142.0, 116335.0, 98849.0]
long_term_incentive: [48521928.0, 1617011.0, 1617011.0, 1294981.0, 974293.0]
other: [42667589.0, 7427621.0, 1852186.0, 465357.0, 425688.0]
salary: [26704229.0, 1060932.0, 492375.0, 428780.0, 415189.0]
Small 5
bonus: [100000.0, 200000.0, 300000.0, 325000.0, 400000.0]
deferral_payments: [51365.0, 53625.0, 204075.0, 260455.0, 504610.0]
deferred_income: [-27992891.0, -3367011.0, -3117011.0, -2334434.0, -1284000.0]
director_fees: [38346.0, 98784.0, 137864.0, 1398517.0]
expenses: [2151.0, 8409.0, 17355.0, 30674.0, 33785.0]
long_term_incentive: [69223.0, 75000.0, 156250.0, 175000.0, 350000.0]
other: [69.0, 540.0, 891.0, 1105.0, 1411.0]
salary: [76399.0, 174246.0, 210500.

In [19]:
##### Trim cash 2: delete 'loan_advances'

cash_sub_df = test_df[cash_li].fillna(value=0)
#cash_sub_df

In [20]:
##### Trim total 1:

test_df['total_payments']    = cash_sub_df.sum(axis = 1)
test_df['total_stock_value'] = stock_sub_df.sum(axis = 1)

In [21]:
test_df = pd.concat([test_df['name'], commu_sub_df, stock_sub_df, cash_sub_df, test_df[total_li]], axis=1).set_index('name')
print(test_df.shape)
test_df.head()

(33, 18)


,from_messages,from_poi_to_this_person,from_this_person_to_poi,shared_receipt_with_poi,to_messages,exercised_stock_options,restricted_stock,restricted_stock_deferred,bonus,deferral_payments,deferred_income,director_fees,expenses,long_term_incentive,other,salary,total_payments,total_stock_value
name,,,,,,,,,,,,,,,,,,
BELDEN TIMOTHY N,484.0,228.0,108.0,5521.0,7991.0,953136.0,157569.0,0.0,5249999.0,2144013.0,-2334434.0,0.0,17355.0,0.0,210698.0,213999.0,5501630.0,1110705.0
BOWEN JR RAYMOND M,27.0,140.0,15.0,1593.0,1858.0,0.0,252055.0,0.0,1350000.0,0.0,-833.0,0.0,65907.0,974293.0,1621.0,278601.0,2669589.0,252055.0
HANNON KEVIN P,32.0,32.0,21.0,1035.0,1045.0,5538001.0,853064.0,0.0,1500000.0,0.0,-3117011.0,0.0,34039.0,1617011.0,11350.0,243293.0,288682.0,6391065.0
DELAINEY DAVID W,3069.0,66.0,609.0,2097.0,3093.0,2291113.0,1323148.0,0.0,3000000.0,0.0,0.0,0.0,86174.0,1294981.0,1661.0,365163.0,4747979.0,3614261.0
CAUSEY RICHARD A,49.0,58.0,12.0,1585.0,1892.0,0.0,2502063.0,0.0,1000000.0,0.0,-235000.0,0.0,30674.0,350000.0,307895.0,415189.0,1868758.0,2502063.0


In [22]:
#####
##### Start Process
#####

In [23]:
##### Import Module

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [24]:
x_mx = x_train_df.to_numpy()
y_mx = y_train_df.to_numpy()
print(np.shape(x_mx))
print(np.shape(y_mx))
print(x_mx[:5])
print(y_mx[:5])

(112, 18)
(112, 1)
[[ 1.80000000e+01  4.20000000e+01  4.00000000e+00  8.64000000e+02
   9.05000000e+02  1.97941750e+07  2.74836400e+06  0.00000000e+00
   1.75000000e+06  0.00000000e+00 -3.50438600e+06  0.00000000e+00
   4.69500000e+04  1.61701100e+06  1.74839000e+05  4.20636000e+05
   5.05050000e+05  2.25425390e+07]
 [ 1.08000000e+02  8.80000000e+01  3.00000000e+01  2.04200000e+03
   3.62700000e+03  1.92500000e+07  6.84367200e+06  0.00000000e+00
   5.60000000e+06  0.00000000e+00  0.00000000e+00  0.00000000e+00
   2.93360000e+04  1.92000000e+06  2.21220000e+04  1.11125800e+06
   8.68271600e+06  2.60936720e+07]
 [ 3.90000000e+01  1.30000000e+01  1.40000000e+01  9.10000000e+01
   2.25000000e+02  1.62439600e+06  8.69220000e+05  0.00000000e+00
   2.00000000e+05  0.00000000e+00 -4.16700000e+03  0.00000000e+00
   2.28840000e+04  0.00000000e+00  1.57332400e+06  2.11844000e+05
   2.00388500e+06  2.49361600e+06]
 [ 4.97937500e+02  6.48000000e+01  4.00923077e+01  1.11136923e+03
   2.15606154e+03 

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x_mx, y_mx, test_size=0.2, random_state=9527)

regr = linear_model.LinearRegression()
regr.fit(x_train, y_train)
y_pred = regr.predict(x_test)

In [26]:
print(f'MSE = {mean_squared_error(y_test, y_pred)}')

MSE = 0.151713687191008


In [27]:
# Lasso
lasso = linear_model.Lasso(alpha=1.0)
lasso.fit(x_train, y_train)
y_pred_2 = lasso.predict(x_test)
print(f'MSE = {mean_squared_error(y_test, y_pred_2)}')

MSE = 0.11494986493772164


In [28]:
##### skip
'''
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
lasso = linear_model.Lasso()
alpha = [0.5, 1.0, 1.5, 2.0, 2.5]
param_grid = dict(alpha=alpha)
grid_search = GridSearchCV(lasso, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)
grid_result = grid_search.fit(x_train, y_train)
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
'''
print('Skip.')

Skip.


In [29]:
unknown_mx = test_df.to_numpy()
print(np.shape(unknown_mx))
print(unknown_mx[:5])

(33, 18)
[[ 4.840000e+02  2.280000e+02  1.080000e+02  5.521000e+03  7.991000e+03
   9.531360e+05  1.575690e+05  0.000000e+00  5.249999e+06  2.144013e+06
  -2.334434e+06  0.000000e+00  1.735500e+04  0.000000e+00  2.106980e+05
   2.139990e+05  5.501630e+06  1.110705e+06]
 [ 2.700000e+01  1.400000e+02  1.500000e+01  1.593000e+03  1.858000e+03
   0.000000e+00  2.520550e+05  0.000000e+00  1.350000e+06  0.000000e+00
  -8.330000e+02  0.000000e+00  6.590700e+04  9.742930e+05  1.621000e+03
   2.786010e+05  2.669589e+06  2.520550e+05]
 [ 3.200000e+01  3.200000e+01  2.100000e+01  1.035000e+03  1.045000e+03
   5.538001e+06  8.530640e+05  0.000000e+00  1.500000e+06  0.000000e+00
  -3.117011e+06  0.000000e+00  3.403900e+04  1.617011e+06  1.135000e+04
   2.432930e+05  2.886820e+05  6.391065e+06]
 [ 3.069000e+03  6.600000e+01  6.090000e+02  2.097000e+03  3.093000e+03
   2.291113e+06  1.323148e+06  0.000000e+00  3.000000e+06  0.000000e+00
   0.000000e+00  0.000000e+00  8.617400e+04  1.294981e+06  1.661

In [30]:
unknown_pred = lasso.predict(unknown_mx)

In [31]:
print(np.shape(unknown_pred))
print(unknown_pred)

(33,)
[ 2.95562593e-01  2.47605525e-01  4.80353815e-01 -4.86300703e-02
  2.24598902e-01  2.19876357e-01  1.04277226e+00  3.11200944e-02
  4.67728457e-01  1.85898399e-01 -1.55197828e-01  4.58872613e-02
  7.95809710e-02  4.99015473e-02  1.56752068e-01  4.77725921e-04
  1.88883208e-01  2.95696833e-01  3.29967594e-01  4.07058140e-02
  1.87811417e+01  2.51326803e-01  3.30386371e-01  5.56797981e-02
  2.15942126e-01  7.57167236e-02 -6.67554738e-01  1.27962211e-01
  1.83673533e-01  1.95702115e-01 -6.62589014e-03  2.76448567e-01
  3.72105946e-01]


In [37]:
unknown_pred_2 = regr.predict(unknown_mx)
print(np.shape(unknown_pred_2))
print(unknown_pred_2)

(33, 1)
[[ 1.70486969e-01]
 [ 2.78522045e-01]
 [ 4.77392402e-01]
 [ 4.75478786e-01]
 [ 2.39527772e-01]
 [ 1.70458789e-01]
 [ 1.30730968e+00]
 [ 6.06635389e-02]
 [ 5.65579559e-01]
 [ 2.21672850e-01]
 [-2.62294977e-01]
 [ 1.08414855e-02]
 [ 9.18964779e-02]
 [ 7.39055020e-02]
 [ 1.24672592e-01]
 [-2.55175025e-02]
 [ 2.47430206e-01]
 [ 3.87186040e-01]
 [ 4.11758778e-01]
 [ 6.47315495e-02]
 [ 2.07879637e+01]
 [ 2.25304631e-01]
 [ 3.65645634e-01]
 [ 7.65403310e-02]
 [ 2.69365190e-01]
 [ 9.24502491e-02]
 [-6.82695892e-01]
 [ 1.60768401e-01]
 [ 1.54648768e-01]
 [ 1.71667965e-01]
 [-2.78118995e-02]
 [ 3.45144604e-01]
 [ 3.49686836e-01]]


In [38]:
unknown_pred_2.min()

-0.6826958921117591

In [39]:
unknown_pred_2.max()

20.78796365301933

In [40]:
unknown_pred_2_nor = (unknown_pred_2 - unknown_pred_2.min()) / (unknown_pred_2.max() - unknown_pred_2.min())
unknown_pred_2_nor

array([[0.03973715],
       [0.04476891],
       [0.05403133],
       [0.0539422 ],
       [0.04295274],
       [0.03973584],
       [0.09268488],
       [0.03462211],
       [0.05813866],
       [0.04212114],
       [0.01958025],
       [0.03230163],
       [0.03607679],
       [0.03523885],
       [0.03760334],
       [0.03060821],
       [0.0433208 ],
       [0.04982995],
       [0.05097443],
       [0.03481157],
       [1.        ],
       [0.04229029],
       [0.0488267 ],
       [0.03536157],
       [0.04434242],
       [0.03610258],
       [0.        ],
       [0.03928451],
       [0.03899948],
       [0.03979216],
       [0.03050134],
       [0.04787186],
       [0.04808342]])

In [42]:
test_df['poi'] = unknown_pred_2_nor
test_df['poi'].to_csv('output/1st_sub.csv', )
#output_df = 